In [ ]:
!pip install -q pickle5
!pip install -q nltk
!pip install -q tensorboardX
!pip install -q transformers
!pip install -q emoji

     |████████████████████████████████| 132 kB 4.3 MB/s 
     |████████████████████████████████| 124 kB 4.3 MB/s 
     |████████████████████████████████| 3.1 MB 4.2 MB/s 
     |████████████████████████████████| 3.3 MB 65.7 MB/s 
     |████████████████████████████████| 596 kB 89.1 MB/s 
     |████████████████████████████████| 59 kB 8.2 MB/s 
     |████████████████████████████████| 895 kB 71.5 MB/s 
     |████████████████████████████████| 170 kB 4.2 MB/s 


In [ ]:
import os
HOME = './drive/MyDrive/tvd5455/VvsNV'
dimension = 'Relations_discrete'
DIMENSION_HOME = './drive/MyDrive/tvd5455/VvsNV/Relations'

### preprocess.py

In [ ]:
from nltk.tokenize import TweetTokenizer
from emoji import demojize
import re
import csv
import numpy as np
import pandas as pd
import pickle5 as pkl
from numba import vectorize,njit

tokenizer = TweetTokenizer()

def normalizeToken(token):
    lowercased_token = token.lower()
    if token.startswith("@"):
        return "@USER"
    elif lowercased_token.startswith("http") or lowercased_token.startswith("www"): 
        return "HTTPURL"
    elif len(token) == 1:
        return demojize(token)
    else:
        if token == "’":
            return "'"
        elif token == "…":
            return "..."
        else:
            return token

def normalizeTweet(tweet):
    tweet = tweet.lower()
    tokens = tokenizer.tokenize(tweet.replace("’", "'").replace("…", "..."))
    normTweet = " ".join([normalizeToken(token) for token in tokens])

    normTweet = normTweet.replace("cannot ", "can not ").replace("n't ", " n't ").replace("n 't ", " n't ").replace("ca n't", "can't").replace("ai n't", "ain't")
    normTweet = normTweet.replace("'m ", " 'm ").replace("'re ", " 're ").replace("'s ", " 's ").replace("'ll ", " 'll ").replace("'d ", " 'd ").replace("'ve ", " 've ")
    normTweet = normTweet.replace(" p . m .", "  p.m.") .replace(" p . m ", " p.m ").replace(" a . m .", " a.m.").replace(" a . m ", " a.m ")

    normTweet = re.sub(r",([0-9]{2,4}) , ([0-9]{2,4})", r",\1,\2", normTweet)
    normTweet = re.sub(r"([0-9]{1,3}) / ([0-9]{2,4})", r"\1/\2", normTweet)
    normTweet = re.sub(r"([0-9]{1,3})- ([0-9]{2,4})", r"\1-\2", normTweet)
    
    return " ".join(normTweet.split())


In [ ]:
verified_file = os.path.join(HOME, 'df_to_label3.pkl')
with open(verified_file, 'rb') as handle:
    df = pkl.load(handle)
    df = df.dropna(subset=['Tweet'])
    print(df.columns)
    print(df.head())
    print(df.shape)
    df["ID"] = df["TweetID"].to_list()
    # df["Text"] = np.vectorize(normalizeTweet)(df["Tweet"])
    df['Text'] = df['Tweet'].apply(lambda x: normalizeTweet(x))
    df = df.dropna(subset=['Text'])

print(df.columns)
print(df.head())
print(df.shape)
df.to_csv(os.path.join(HOME, 'common_data/df_to_label3.csv'), quoting=csv.QUOTE_NONNUMERIC)

### make_data.py

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import os

goldfile =  os.path.join(HOME, 'Gold_Standard.xlsx')


# ### Make Train and Test dataset
train_df = pd.read_excel(goldfile)

# for consistency with unlabeled data
train_df["ID"] = train_df["TWEETID"].to_list()
train_df["Text"] = train_df["Raw_Tweet"].apply(lambda x: normalizeTweet(x))
# train_df["Text"] = list(np.vectorize(normalizeTweet)(train_df["Raw_Tweet"]))

train_df, test_df = train_test_split(train_df, test_size=0.20, random_state=47)
train_df, dev_df = train_test_split(train_df, test_size=0.20, random_state=47)

train_df.to_csv(os.path.join(HOME, "common_data/train.csv"))
dev_df.to_csv(os.path.join(HOME, "common_data/dev.csv"))
test_df.to_csv(os.path.join(HOME, "common_data/test.csv"))

In [ ]:
import os
import sys
import logging
import pandas as pd
import pickle
from tqdm import tqdm, trange
from multiprocessing import Pool, cpu_count

logger = logging.getLogger()


class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text, binary_label=None, level_label=None):
        """Constructs a InputExample.
        Args:
            guid: Unique id for the example.
            text: string. The untokenized text of the tweet sentences. 
            label: (Optional) int. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text = text
        self.binary_label = binary_label
        self.level_label = level_label


class DataProcessor(object):
    """Base class for data converters for sequence classification data sets."""

    def get_train_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the dev set."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

    @classmethod
    def _read_pkl(cls, input_file, quotechar=None):
        """Reads a tab separated value file."""
        with open(input_file, 'rb') as handle:
            return pickle.load(handle)
    
    @classmethod
    def _read_csv(cls, input_file, quotechar=None):
        """Reads a tab separated value file."""
        return pd.read_csv(input_file)


class BinaryClassificationProcessor(DataProcessor):
    """Processor for binary classification dataset."""

    def get_train_examples(self, data_dir, dimension):
        """See base class."""
        return self._create_examples(
            self._read_csv(os.path.join(data_dir, "train.csv")), "train", dimension)

    def get_dev_examples(self, data_dir, dimension):
        """See base class."""
        return self._create_examples(
            self._read_csv(os.path.join(data_dir, "dev.csv")), "dev", dimension)

    def get_test_examples(self, data_dir, dimension):
        """See base class."""
        return self._create_examples(
            self._read_csv(os.path.join(data_dir, "test.csv")), "test", dimension)
    
    def get_unlabeled_examples(self, data_dir, dimension, filename):
        """See base class."""
        return self._create_examples(
            self._read_csv(os.path.join(data_dir, filename)), "unlabeled", dimension)
    
    def get_unlabeled_pickles(self, data_dir):
        return self._create_examples_from_pkl(
            self._read_pkl(os.path.join(data_dir, "tweet_txt.pkl")), "unlabeled")

    def get_labels(self):
        """See base class."""
        return [0, 1]

    def _preprocess_label(self, label):
        binary_label, level_label = 0, 0
        if label > 1:
            binary_label = 1
            level_label = (label - 4) / 3
        return binary_label, level_label
    
    def _create_examples(self, tweets_list, set_type, dimension):
        """Creates examples for the training and dev sets."""
        assert "discrete" in dimension
        examples = []
        # for _, tweet in tqdm(tweets_list.iterrows(), desc="Creating examples"):
        for _, tweet in tweets_list.iterrows():
            guid = tweet['ID']
            text = tweet['Text']
            # if 'Information_discrete' in tweet:
            #     label = tweet['Information_discrete']
            #     binary_label, level_label = self._preprocess_label(label)
            #     example = InputExample(guid, text, binary_label, level_label)
            if dimension in tweet:
                label = tweet[dimension]
                binary_label, level_label = self._preprocess_label(label)
                example = InputExample(guid, text, binary_label, level_label)
            else:
                example = InputExample(guid, text)
            examples.append(example)
        
        return examples
    
    def _create_examples_from_pkl(self, tweets_dict, set_type):
        """Creates examples for the training and dev sets."""
        examples = []
        
        for guid, text in tweets_dict.items():
            example = InputExample(guid, text)
            examples.append(example)
        
        return examples


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, binary_label, level_label):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.binary_label = binary_label
        self.level_label = level_label


def convert_example_to_feature(example_row):
    # return example_row
    example, label_map, max_seq_length, tokenizer, output_mode = example_row
    
    # text = normalizeTweet(example.text)
    text = example.text
    tokens = tokenizer.tokenize(text)

    # Account for [CLS] and [SEP] with "- 2"
    if len(tokens) > max_seq_length - 2:
        tokens = tokens[:(max_seq_length - 2)]

    tokens = [tokenizer.cls_token] + tokens + [tokenizer.sep_token] 
    segment_ids = [0] * len(tokens) 

#     ####
#     text = "I disclosed my personal information."
#     tokens_q = tokenizer.tokenize(text)

#     # Account for [SEP] with "- 1"
#     if len(tokens) > 50 - 1:
#         tokens_q = tokens_q[:(50 - 1)]

#     tokens = tokens + tokens_q + [tokenizer.sep_token] 
#     segment_ids += [1] * (len(tokens_q) + 1)
#     ####
    
    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1] * len(input_ids)

    # Zero-pad up to the sequence length.
#     padding = [0] * (max_seq_length + 50 - len(input_ids))
    padding = [0] * (max_seq_length - len(input_ids))
    input_ids += padding
    input_mask += padding
    segment_ids += padding

    # if len(input_ids) != (max_seq_length):
        # pdb.set_trace()
    
    assert len(input_ids) == (max_seq_length)
    assert len(input_mask) == (max_seq_length)
    assert len(segment_ids) == (max_seq_length)

    binary_label = example.binary_label
    level_label = example.level_label
    
    feature = InputFeatures(input_ids=input_ids,
                 input_mask=input_mask,
                 segment_ids=segment_ids,
                 binary_label=binary_label,
                 level_label=level_label)
    
    return feature


def convert_examples_to_features(examples, label_list, max_seq_length, tokenizer, output_mode):
    label_map = {label: i for i, label in enumerate(label_list)}
    
    examples = [(example, label_map, max_seq_length, tokenizer, output_mode) for example in examples]
    
#     process_count = cpu_count() - 2
    
#     with Pool(process_count) as p:
#         features = list(tqdm(p.imap(convert_example_to_feature, examples, chunksize=100), total=len(examples)))
#         # features = list(p.imap(convert_example_to_feature, examples, chunksize=100))
    
    features = [convert_example_to_feature(ex) for ex in examples]
    
    return features

### classification.py

In [ ]:
import math
import numpy as np
import torch
import json
from torch.utils.data import (DataLoader, WeightedRandomSampler, SequentialSampler, TensorDataset)
from torch import nn
from torch.nn import CrossEntropyLoss, MSELoss, ReLU
import torch.nn.functional as F

from tqdm import tqdm, trange
from tensorboardX import SummaryWriter
import os
from transformers import (WEIGHTS_NAME, BertConfig, BertModel, BertTokenizer,
                                  XLMConfig, XLMModel, XLMTokenizer, 
                                  XLNetConfig, XLNetModel, XLNetTokenizer,
                                  RobertaConfig, RobertaModel, RobertaTokenizer)
from transformers import AdamW, get_linear_schedule_with_warmup

def load_and_cache_examples(task, tokenizer, evaluate=False, final=False):
    processor = processors[task]()
    output_mode = args['output_mode']
    
    mode = 'dev' if (evaluate and not final) else ('test' if (evaluate and final) else 'train')
    cached_features_file = os.path.join(args['processed_data_dir'], f"cached_{mode}_{args['model_name']}_{args['max_seq_length']}_{task}")
    
    if os.path.exists(cached_features_file) and not args['reprocess_input_data']:
        logger.info("Loading features from cached file %s", cached_features_file)
        features = torch.load(cached_features_file)
               
    else:
        logger.info("Creating features from dataset file at %s", args['data_dir'])
        label_list = processor.get_labels()
        examples = processor.get_dev_examples(args['data_dir'], dimension=dimension) if (evaluate and not final) else\
                    (processor.get_test_examples(args['data_dir'], dimension=dimension) if (evaluate and final) else\
                     processor.get_train_examples(args['data_dir'], dimension=dimension))
        
        features = convert_examples_to_features(examples, label_list, args['max_seq_length'], tokenizer, output_mode)
        
        filedir = args['processed_data_dir']
        if not os.path.exists(filedir):
            os.makedirs(filedir)
        logger.info("Saving features into cached file %s", cached_features_file)
        torch.save(features, cached_features_file)
    
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
    if output_mode == "both":
        all_binary_labels = torch.tensor([f.binary_label for f in features], dtype=torch.long)
        all_level_labels = torch.tensor([f.level_label for f in features], dtype=torch.float)
    else:
        raise KeyError(output_mode)

    dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_binary_labels, all_level_labels)
    return dataset

In [ ]:
processors = {
    "both": BinaryClassificationProcessor
}

output_modes = {
    "both": "classification|regression"
}

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)


args = {
    'data_dir': os.path.join(HOME, 'common_data/'),
    'processed_data_dir': os.path.join(DIMENSION_HOME, 'processed_data/'),
    'model_type':  'roberta',
    'model_name': 'roberta-large',
    'task_name': 'both',
    'output_dir': os.path.join(DIMENSION_HOME, 'outputs_roberta/'),
    'cache_dir': os.path.join(DIMENSION_HOME,'cache/'),
    'do_train': True,
    'do_eval': True,
    'fp16': False,
    'fp16_opt_level': 'O1',
    'max_seq_length': 128,
    'output_mode': 'both',
    'train_batch_size': 16,
    'eval_batch_size': 128,

    'gradient_accumulation_steps': 1,
    'num_train_epochs': 4,
    'weight_decay': 0,
    'learning_rate': 1e-5,
    'adam_epsilon': 1e-8,
    'warmup_ratio': 0.06,
    'warmup_steps': 0,
    'max_grad_norm': 1.0,

    'logging_steps': 100,
    'evaluate_during_training': True,
    'save_steps': 200,
    'eval_all_checkpoints': True,

    'overwrite_output_dir': True,
    'reprocess_input_data': True,
    'notes': 'Using Twitter disclosure dataset'
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


with open(os.path.join(HOME, 'args.json'), 'w') as f:
    json.dump(args, f)


if os.path.exists(args['output_dir']) and os.listdir(args['output_dir']) and args['do_train'] and not args['overwrite_output_dir']:
    raise ValueError("Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(args['output_dir']))



MODEL_CLASSES = {
    'bert': (BertConfig, BertModel, BertTokenizer),
    'xlnet': (XLNetConfig, XLNetModel, XLNetTokenizer),
    'xlm': (XLMConfig, XLMModel, XLMTokenizer),
    'roberta': (RobertaConfig, RobertaModel, RobertaTokenizer)
}

config_class, model_class, tokenizer_class = MODEL_CLASSES[args['model_type']]


config = config_class.from_pretrained(args['model_name'], num_labels=2, finetuning_task=args['task_name'])
tokenizer = tokenizer_class.from_pretrained(args['model_name'])


base_model = model_class.from_pretrained(args['model_name'])

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
@torch.jit.script
def mish(input):
    return input * torch.tanh(F.softplus(input))
  
class Mish(nn.Module):
    def forward(self, input):
        return mish(input)


class ClassificationHead(nn.Module):
    """Head for sentence-level classification tasks."""

    def __init__(self, hidden_size, dropout, num_labels):
        super().__init__()
        # self.dense = nn.Linear(hidden_size, 192)
        self.dropout = nn.Dropout(dropout)
        self.out_proj = nn.Linear(hidden_size, num_labels)

    def forward(self, features, **kwargs):
        x = features
        # x = self.dropout(x)
        # x = self.dense(x)
        # x = torch.relu(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x
    

class ClassificationModel(nn.Module):
    def __init__(self, base_model, base_model_output_size=1024, dropout=0.1):
        super().__init__()
        self.base_model = base_model
        
        self.dropout = nn.Dropout(dropout)
        self._lambda = 1.0
        self.classifier = ClassificationHead(base_model_output_size, dropout, 2)
        self.regressor = nn.Linear(base_model_output_size, 1)
        # for param in self.base_model.parameters():
        #     param.requires_grad = False

    def forward(self, input_ids, attention_mask, token_type_ids, binary_labels=None, level_labels=None, do_binary=True):
        X, attention_mask = input_ids, attention_mask
        outputs = self.base_model(X, attention_mask=attention_mask, token_type_ids=token_type_ids)
        
        pooled_output = outputs[1]

        # pooled_output = self.dropout(pooled_output)
        binary_logits = self.classifier(pooled_output)
        level_logit = self.regressor(pooled_output)

        if binary_labels is not None:
            outputs = (binary_logits, level_logit,) + outputs[2:]  # add hidden states and attention if they are here
            
            binary_loss_fct = CrossEntropyLoss()
            binary_loss = binary_loss_fct(binary_logits.view(-1, 2), binary_labels.view(-1))
            
            regression_loss_fct = MSELoss()
#             regression_loss = regression_loss_fct(
#                 level_logit[binary_labels==1, :].view(-1), 
#                 level_labels[binary_labels==1].view(-1)
#             )
            regression_loss = 0
            # pdb.set_trace()
            print("\r{:.5f} {:.5f}".format(binary_loss, regression_loss), end='')
            loss = binary_loss + self._lambda * regression_loss
            
            outputs = (loss,) + outputs
        else:
            level_logits = torch.clamp(level_logit, min=0, max=1)
            outputs = (binary_logits, level_logit,) + outputs[2:]  # add hidden states and attention if they are here

        return outputs  # (loss), binary_logits, level_logits, (hidden_states), (attentions)



In [ ]:
model = ClassificationModel(base_model=base_model)
model.to(device);


task = args['task_name']

if task in processors.keys() and task in output_modes.keys():
    processor = processors[task]()
    label_list = processor.get_labels()
    num_labels = len(label_list)
else:
    raise KeyError(f'{task} not found in processors or in output_modes. Please check utils.py.')


In [ ]:
def get_weights_for_sampler(dataset):
    sampler = SequentialSampler(dataset)
    dataloader = DataLoader(dataset, sampler=sampler, batch_size=len(dataset))
    batch = None
    for _batch in dataloader:
        batch = _batch
    labels = batch[3].numpy()
    num_neg = len(labels[labels == 0])
    num_pos = len(labels[labels == 1])
    weights = [1./num_neg if l == 0 else 1./num_pos for l in labels]
    return weights
    

def train(train_dataset, model, tokenizer):
    tb_writer = SummaryWriter(log_dir=os.path.join(DIMENSION_HOME, 'runs'))
    
    weights = get_weights_for_sampler(train_dataset)
    train_sampler = WeightedRandomSampler(num_samples=len(train_dataset), weights=weights)
    train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args['train_batch_size'])
    
    t_total = len(train_dataloader) // args['gradient_accumulation_steps'] * args['num_train_epochs']
    
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': args['weight_decay']},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
    
    warmup_steps = math.ceil(t_total * args['warmup_ratio'])
    args['warmup_steps'] = warmup_steps if args['warmup_steps'] == 0 else args['warmup_steps']
    
    optimizer = AdamW(optimizer_grouped_parameters, lr=args['learning_rate'], eps=args['adam_epsilon'])
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=args['warmup_steps'], num_training_steps=t_total*2)
    
    if args['fp16']:
        try:
            from apex import amp
        except ImportError:
            raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
        model, optimizer = amp.initialize(model, optimizer, opt_level=args['fp16_opt_level'])
        
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args['num_train_epochs'])
    logger.info("  Total train batch size  = %d", args['train_batch_size'])
    logger.info("  Gradient Accumulation steps = %d", args['gradient_accumulation_steps'])
    logger.info("  Total optimization steps = %d", t_total)

    global_step = 0
    tr_loss, logging_loss = 0.0, 0.0
    model.zero_grad()
    train_iterator = trange(int(args['num_train_epochs']), desc="Epoch")
    
    for ee in train_iterator:
        logger.info('\n\n')
        epoch_iterator = tqdm(train_dataloader, desc="Iteration")
        for step, batch in enumerate(epoch_iterator):
            model.train()
            batch = tuple(t.to(device) for t in batch)
            inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                      'token_type_ids': batch[2] if args['model_type'] in ['bert', 'xlnet'] else None,  # XLM don't use segment_ids
                      'binary_labels':  batch[3],
                      'level_labels':   batch[4],
                      'do_binary':      ee == 0}
            outputs = model(**inputs)
            loss = outputs[0]  # model outputs are always tuple in pytorch-transformers (see doc)
            # print("\r%f" % loss, end='')

            if args['gradient_accumulation_steps'] > 1:
                loss = loss / args['gradient_accumulation_steps']

            if args['fp16']:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
                torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args['max_grad_norm'])
                
            else:
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), args['max_grad_norm'])

            tr_loss += loss.item()
            if (step + 1) % args['gradient_accumulation_steps'] == 0:
                optimizer.step()
                scheduler.step()
                model.zero_grad()
                global_step += 1

                if args['logging_steps'] > 0 and global_step % args['logging_steps'] == 0:
                    # Log metrics
                    if args['evaluate_during_training']:  # Only evaluate when single GPU otherwise metrics may not average well
                        results, _ = evaluate(model, tokenizer)
                        for key, value in results.items():
                            tb_writer.add_scalar('eval_{}'.format(key), value, global_step)
                    tb_writer.add_scalar('lr', scheduler.get_lr()[0], global_step)
                    tb_writer.add_scalar('loss', (tr_loss - logging_loss)/args['logging_steps'], global_step)
                    logging_loss = tr_loss

                if args['save_steps'] > 0 and global_step % args['save_steps'] == 0:
                    # Save model checkpoint
                    output_dir = os.path.join(args['output_dir'], 'checkpoint-{}'.format(global_step))
                    if not os.path.exists(output_dir):
                        os.makedirs(output_dir)
                    model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
                    # model_to_save.save_pretrained(output_dir)
                    torch.save(model_to_save.state_dict(), output_dir + '/' + WEIGHTS_NAME)
                    logger.info("Saving model checkpoint to %s", output_dir)


    return global_step, tr_loss / global_step

In [ ]:
from sklearn.metrics import mean_squared_error, matthews_corrcoef, confusion_matrix, f1_score
from scipy.stats import pearsonr

def get_mismatched(labels, preds):
    mismatched = labels != preds
    examples = processor.get_dev_examples(args['data_dir'], dimension=dimension)
    wrong = [i for (i, v) in zip(examples, mismatched) if v]
    
    return wrong

def get_eval_report(labels, preds):
    mcc = matthews_corrcoef(labels, preds)
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    f1 = f1_score(labels, preds)
    return {
        "mcc": mcc,
        "f1": f1,
        "tp": tp,
        "tn": tn,
        "fp": fp,
        "fn": fn
    }, get_mismatched(labels, preds)

def compute_metrics(task_name, preds, labels):
    assert len(preds) == len(labels)
    return get_eval_report(labels, preds)

def get_eval_report_reg(labels, preds):
    rmse = mean_squared_error(labels, preds, squared=False)
    return {
        "rmse": rmse
    }

def compute_regression_metrics(task_name, preds, labels):
    assert len(preds) == len(labels)
    return get_eval_report_reg(labels, preds)

def evaluate(model, tokenizer, prefix="", final=False):
    model.eval()
    
    # Loop to handle MNLI double evaluation (matched, mis-matched)
    eval_output_dir = args['output_dir']

    results = {}
    EVAL_TASK = args['task_name']

    eval_dataset = load_and_cache_examples(EVAL_TASK, tokenizer, evaluate=True, final=final)
    if not os.path.exists(eval_output_dir):
        os.makedirs(eval_output_dir)


    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=args['eval_batch_size'])

    # Eval!
    logger.info("***** Running evaluation {} *****".format(prefix))
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", args['eval_batch_size'])
    eval_loss = 0.0
    nb_eval_steps = 0
    binary_preds = None
    level_preds = None
    out_binary_labels = None
    out_level_labels = None
    # for batch in tqdm(eval_dataloader, desc="Evaluating"):
    for batch in eval_dataloader:
        batch = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                      'token_type_ids': batch[2] if args['model_type'] in ['bert', 'xlnet'] else None,  # XLM don't use segment_ids
                      'binary_labels':  batch[3],
                      'level_labels':   batch[4]}
            outputs = model(**inputs)
            tmp_eval_loss, binary_logits, level_logit = outputs[:3]

            eval_loss += tmp_eval_loss.mean().item()
        nb_eval_steps += 1
        if binary_preds is None:
            binary_preds = binary_logits.detach().cpu().numpy()
            out_binary_labels = inputs['binary_labels'].detach().cpu().numpy()
            
            level_preds = level_logit.detach().cpu().numpy()
            out_level_labels = inputs['level_labels'].detach().cpu().numpy()
        
        else:
            binary_preds = np.append(binary_preds, binary_logits.detach().cpu().numpy(), axis=0)
            out_binary_labels = np.append(out_binary_labels, inputs['binary_labels'].detach().cpu().numpy(), axis=0)
        
            level_preds = np.append(level_preds, level_logit.detach().cpu().numpy(), axis=0)
            out_level_labels = np.append(out_level_labels, inputs['level_labels'].detach().cpu().numpy(), axis=0)

    eval_loss = eval_loss / nb_eval_steps
    
    binary_preds = np.argmax(binary_preds, axis=1)
    level_preds = np.squeeze(level_preds)
    
    result, wrong = compute_metrics(EVAL_TASK, binary_preds, out_binary_labels)
    results.update(result)
    
    # pdb.set_trace()
    
    result = compute_regression_metrics(EVAL_TASK, level_preds[out_binary_labels==1], out_level_labels[out_binary_labels==1])
    results = dict(list(results.items()) + list(result.items()))

    output_eval_file = os.path.join(eval_output_dir, "eval_results.txt")
    with open(output_eval_file, "w") as writer:
        logger.info("***** Eval results {} *****".format(prefix))
        for key in sorted(results.keys()):
            logger.info("  %s = %s", key, str(results[key]))
            writer.write("%s = %s\n" % (key, str(results[key])))

    return results, wrong

In [ ]:
if args['do_train']:
    train_dataset = load_and_cache_examples(task, tokenizer)
    global_step, tr_loss = train(train_dataset, model, tokenizer)
    logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)



if args['do_train']:
    if not os.path.exists(args['output_dir']):
        os.makedirs(args['output_dir'])
    logger.info("Saving model checkpoint to %s", args['output_dir'])
    
    model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
    # model_to_save.save_pretrained(args['output_dir'])
    output_dir = os.path.join(args['output_dir'], 'checkpoint-{}'.format(global_step))
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    torch.save(model_to_save.state_dict(), output_dir + '/' + WEIGHTS_NAME)
    tokenizer.save_pretrained(args['output_dir'])
    torch.save(args, os.path.join(args['output_dir'], 'training_args.bin'))

INFO:root:Creating features from dataset file at ./drive/MyDrive/tvd5455/VvsNV/common_data/
INFO:root:Saving features into cached file ./drive/MyDrive/tvd5455/VvsNV/Relations/processed_data/cached_train_roberta-large_128_both
INFO:root:***** Running training *****
INFO:root:  Num examples = 3200
INFO:root:  Num Epochs = 4
INFO:root:  Total train batch size  = 16
INFO:root:  Gradient Accumulation steps = 1
INFO:root:  Total optimization steps = 800
Epoch:   0%|          | 0/4 [00:00<?, ?it/s]INFO:root:



Iteration:   0%|          | 0/200 [00:00<?, ?it/s]

0.69529 0.00000


Iteration:   0%|          | 1/200 [00:00<02:40,  1.24it/s]

0.68920 0.00000


Iteration:   1%|          | 2/200 [00:01<02:32,  1.30it/s]

...

INFO:root:Creating features from dataset file at ./drive/MyDrive/tvd5455/VvsNV/common_data/
INFO:root:Saving features into cached file ./drive/MyDrive/tvd5455/VvsNV/Relations/processed_data/cached_dev_roberta-large_128_both
INFO:root:***** Running evaluation  *****
INFO:root:  Num examples = 800
INFO:root:  Batch size = 128


0.44066 0.00000

INFO:root:***** Eval results  *****
INFO:root:  f1 = 0.39999999999999997
INFO:root:  fn = 27
INFO:root:  fp = 24
INFO:root:  mcc = 0.3666356669338706
INFO:root:  rmse = 0.4598583
INFO:root:  tn = 732
INFO:root:  tp = 17


1.31272 0.00000

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:247: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
INFO:root:Saving model checkpoint to ./drive/MyDrive/tvd5455/VvsNV/Relations/outputs_roberta/checkpoint-800

Epoch: 100%|██████████| 4/4 [11:38<00:00, 174.62s/it]
INFO:root: global_step = 800, average loss = 0.1612196562799545
INFO:root:Saving model checkpoint to ./drive/MyDrive/tvd5455/VvsNV/Relations/outputs_roberta/


In [ ]:
import glob

results = {}
if args['do_eval']:
    print ('\n\n')
    checkpoints = [args['output_dir']]
    if args['eval_all_checkpoints']:
        # checkpoints = list(os.path.dirname(c) for c in sorted(glob.glob(args['output_dir'] + '/**/' + WEIGHTS_NAME, recursive=True)))
        checkpoints = list(c for c in sorted(glob.glob(args['output_dir'] + '/**/' + WEIGHTS_NAME, recursive=True)))
        logging.getLogger("pytorch_transformers.modeling_utils").setLevel(logging.WARN)  # Reduce logging
    logger.info("Evaluate the following checkpoints: %s", checkpoints)
    for checkpoint in checkpoints:
        global_step = checkpoint.split('-')[-1] if len(checkpoints) > 1 else ""
        model = ClassificationModel(base_model=base_model)
        model.load_state_dict(torch.load(checkpoint))
        model.to(device)
        result, wrong_preds = evaluate(model, tokenizer, prefix=global_step, final=True)
        result = dict((k + '_{}'.format(global_step), v) for k, v in result.items())
        results.update(result)
        
print(results)

eval_output_dir = args['output_dir']
output_test_file = os.path.join(eval_output_dir, "test_results.txt")
with open(output_test_file, "w") as writer:
    for key in sorted(results.keys()):
        writer.write("%s = %s\n" % (key, str(results[key])))

INFO:root:Evaluate the following checkpoints: ['./drive/MyDrive/tvd5455/VvsNV/Relations/outputs_roberta/checkpoint-200/pytorch_model.bin', './drive/MyDrive/tvd5455/VvsNV/Relations/outputs_roberta/checkpoint-400/pytorch_model.bin', './drive/MyDrive/tvd5455/VvsNV/Relations/outputs_roberta/checkpoint-600/pytorch_model.bin', './drive/MyDrive/tvd5455/VvsNV/Relations/outputs_roberta/checkpoint-800/pytorch_model.bin']


INFO:root:Creating features from dataset file at ./drive/MyDrive/tvd5455/VvsNV/common_data/
INFO:root:Saving features into cached file ./drive/MyDrive/tvd5455/VvsNV/Relations/processed_data/cached_test_roberta-large_128_both
INFO:root:***** Running evaluation 200/pytorch_model.bin *****
INFO:root:  Num examples = 1000
INFO:root:  Batch size = 128


0.38092 0.00000

INFO:root:***** Eval results 200/pytorch_model.bin *****
INFO:root:  f1 = 0.5420560747663552
INFO:root:  fn = 6
INFO:root:  fp = 43
INFO:root:  mcc = 0.557416815155433
INFO:root:  rmse = 0.4328605
INFO:root:  tn = 922
INFO:root:  tp = 29
INFO:root:Creating features from dataset file at ./drive/MyDrive/tvd5455/VvsNV/common_data/
INFO:root:Saving features into cached file ./drive/MyDrive/tvd5455/VvsNV/Relations/processed_data/cached_test_roberta-large_128_both
INFO:root:***** Running evaluation 400/pytorch_model.bin *****
INFO:root:  Num examples = 1000
INFO:root:  Batch size = 128


0.05781 0.00000

...

0.36959 0.00000

INFO:root:Creating features from dataset file at ./drive/MyDrive/tvd5455/VvsNV/common_data/
INFO:root:Saving features into cached file ./drive/MyDrive/tvd5455/VvsNV/Relations/processed_data/cached_test_roberta-large_128_both
INFO:root:***** Running evaluation 800/pytorch_model.bin *****
INFO:root:  Num examples = 1000
INFO:root:  Batch size = 128


0.05224 0.00000

INFO:root:***** Eval results 800/pytorch_model.bin *****
INFO:root:  f1 = 0.42857142857142855
INFO:root:  fn = 20
INFO:root:  fp = 20
INFO:root:  mcc = 0.4078460399703923
INFO:root:  rmse = 0.35311806
INFO:root:  tn = 945
INFO:root:  tp = 15


0.35635 0.00000{'mcc_200/pytorch_model.bin': 0.557416815155433, 'f1_200/pytorch_model.bin': 0.5420560747663552, 'tp_200/pytorch_model.bin': 29, 'tn_200/pytorch_model.bin': 922, 'fp_200/pytorch_model.bin': 43, 'fn_200/pytorch_model.bin': 6, 'rmse_200/pytorch_model.bin': 0.4328605, 'mcc_400/pytorch_model.bin': 0.47525261556040793, 'f1_400/pytorch_model.bin': 0.4883720930232558, 'tp_400/pytorch_model.bin': 21, 'tn_400/pytorch_model.bin': 935, 'fp_400/pytorch_model.bin': 30, 'fn_400/pytorch_model.bin': 14, 'rmse_400/pytorch_model.bin': 0.3647765, 'mcc_600/pytorch_model.bin': 0.4719123366108799, 'f1_600/pytorch_model.bin': 0.4878048780487805, 'tp_600/pytorch_model.bin': 20, 'tn_600/pytorch_model.bin': 938, 'fp_600/pytorch_model.bin': 27, 'fn_600/pytorch_model.bin': 15, 'rmse_600/pytorch_model.bin': 0.34483138, 'mcc_800/pytorch_model.bin': 0.4078460399703923, 'f1_800/pytorch_model.bin': 0.42857142857142855, 'tp_800/pytorch_model.bin': 15, 'tn_800/pytorch_model.bin': 945, 'fp_800/pytorch_mode

### predict.py

In [ ]:
import math
import numpy as np
import torch
import json
from torch.utils.data import (DataLoader, WeightedRandomSampler, SequentialSampler, TensorDataset)
from torch import nn
from torch.nn import CrossEntropyLoss, MSELoss, ReLU
import torch.nn.functional as F

from tqdm import tqdm, trange
from tensorboardX import SummaryWriter
import os
from transformers import (WEIGHTS_NAME, BertConfig, BertModel, BertTokenizer,
                                  XLMConfig, XLMModel, XLMTokenizer, 
                                  XLNetConfig, XLNetModel, XLNetTokenizer,
                                  RobertaConfig, RobertaModel, RobertaTokenizer)
from transformers import AdamW, get_linear_schedule_with_warmup


def load_and_cache_examples(task, tokenizer, evaluate=False, final=False):
    processor = processors[task]()
    output_mode = args['output_mode']
    
    mode = 'df_to_label3'
    cached_features_file = os.path.join(args['processed_data_dir'], f"cached_{mode}_{args['model_name']}_{args['max_seq_length']}_{task}")
    
    if os.path.exists(cached_features_file) and not args['reprocess_input_data']:
        logger.info("Loading features from cached file %s", cached_features_file)
        features = torch.load(cached_features_file)
               
    else:
        logger.info("Creating features from dataset file at %s", args['data_dir'])
        label_list = processor.get_labels()
        examples = processor.get_unlabeled_examples(args['data_dir'], dimension=dimension, filename='df_to_label3.csv')
        logger.info(f"There are {len(examples)} examples....")
        logger.info(examples[0].text)
        features = convert_examples_to_features(examples, label_list, args['max_seq_length'], tokenizer, output_mode)
        
        logger.info("Saving features into cached file %s", cached_features_file)
        torch.save(features, cached_features_file)
    
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
    
    dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids)
    return dataset

In [ ]:
processors = {
    "both": BinaryClassificationProcessor
}

output_modes = {
    "both": "classification|regression"
}

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)


args = {
    'data_dir': os.path.join(HOME, 'common_data/'),
    'processed_data_dir': os.path.join(DIMENSION_HOME, 'processed_data/'),
    'model_type':  'roberta',
    # 'model_name': 'roberta-large-mnli',
    'model_name': 'roberta-large',
    'task_name': 'both',
    'output_dir': os.path.join(DIMENSION_HOME, 'outputs_roberta/'),
    'cache_dir': os.path.join(DIMENSION_HOME, 'cache/'),
    'do_train': False,
    'do_eval': False,
    'do_predict': True,
    'fp16': False,
    'fp16_opt_level': 'O1',
    'max_seq_length': 128,
    'output_mode': 'both',
    'train_batch_size': 16,
    'eval_batch_size': 150,

    'gradient_accumulation_steps': 1,
    'num_train_epochs': 4,
    'weight_decay': 0,
    'learning_rate': 1e-5,
    'adam_epsilon': 1e-8,
    'warmup_ratio': 0.06,
    'warmup_steps': 0,
    'max_grad_norm': 1.0,

    'logging_steps': 100,
    'evaluate_during_training': True,
    'save_steps': 200,
    'eval_all_checkpoints': True,

    'overwrite_output_dir': True,
    'reprocess_input_data': False,
    'notes': 'Using Twitter disclosure dataset'
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
with open(os.path.join(DIMENSION_HOME, 'args.json'), 'w') as f:
    json.dump(args, f)


if os.path.exists(args['output_dir']) and os.listdir(args['output_dir']) and args['do_train'] and not args['overwrite_output_dir']:
    raise ValueError("Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(args['output_dir']))



MODEL_CLASSES = {
    'bert': (BertConfig, BertModel, BertTokenizer),
    'xlnet': (XLNetConfig, XLNetModel, XLNetTokenizer),
    'xlm': (XLMConfig, XLMModel, XLMTokenizer),
    'roberta': (RobertaConfig, RobertaModel, RobertaTokenizer)
}

config_class, model_class, tokenizer_class = MODEL_CLASSES[args['model_type']]


config = config_class.from_pretrained(args['model_name'], num_labels=2, finetuning_task=args['task_name'])
tokenizer = tokenizer_class.from_pretrained(args['model_name'])


base_model = model_class.from_pretrained(args['model_name'])

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
@torch.jit.script
def mish(input):
    return input * torch.tanh(F.softplus(input))
  
class Mish(nn.Module):
    def forward(self, input):
        return mish(input)


class ClassificationHead(nn.Module):
    """Head for sentence-level classification tasks."""

    def __init__(self, hidden_size, dropout, num_labels):
        super().__init__()
        # self.dense = nn.Linear(hidden_size, 192)
        self.dropout = nn.Dropout(dropout)
        self.out_proj = nn.Linear(hidden_size, num_labels)

    def forward(self, features, **kwargs):
        x = features
        # x = self.dropout(x)
        # x = self.dense(x)
        # x = torch.relu(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x
    

class ClassificationModel(nn.Module):
    def __init__(self, base_model, base_model_output_size=1024, dropout=0.1):
        super().__init__()
        self.base_model = base_model
        
        self.dropout = nn.Dropout(dropout)
        self._lambda = 1.0
        self.classifier = ClassificationHead(base_model_output_size, dropout, 2)
        self.regressor = nn.Linear(base_model_output_size, 1)
        # for param in self.base_model.parameters():
        #     param.requires_grad = False

    def forward(self, input_ids, attention_mask, token_type_ids, binary_labels=None, level_labels=None, do_binary=True):
        X, attention_mask = input_ids, attention_mask
        outputs = self.base_model(X, attention_mask=attention_mask, token_type_ids=token_type_ids)
        
        pooled_output = outputs[1]

        # pooled_output = self.dropout(pooled_output)
        binary_logits = self.classifier(pooled_output)
        # level_logit = self.regressor(pooled_output)
        level_logit = None

        if binary_labels is not None:
            outputs = (binary_logits, level_logit,) + outputs[2:]  # add hidden states and attention if they are here
            
            binary_loss_fct = CrossEntropyLoss()
            binary_loss = binary_loss_fct(binary_logits.view(-1, 2), binary_labels.view(-1))
            
            regression_loss_fct = MSELoss()
#             regression_loss = regression_loss_fct(
#                 level_logit[binary_labels==1, :].view(-1), 
#                 level_labels[binary_labels==1].view(-1)
#             )
            regression_loss = 0
            # pdb.set_trace()
            print("\r{:.5f} {:.5f}".format(binary_loss, regression_loss), end='')
            loss = binary_loss + self._lambda * regression_loss
            
            outputs = (loss,) + outputs
        else:
            # level_logits = torch.clamp(level_logit, min=0, max=1)
            outputs = (binary_logits, level_logit,) + outputs[2:]  # add hidden states and attention if they are here

        return outputs  # (loss), binary_logits, level_logits, (hidden_states), (attentions)


In [ ]:
# model = ClassificationModel(base_model=base_model)
# model.to(device);

task = args['task_name']

if task in processors.keys() and task in output_modes.keys():
    processor = processors[task]()
    label_list = processor.get_labels()
    num_labels = len(label_list)
else:
    raise KeyError(f'{task} not found in processors or in output_modes. Please check utils.py.')




In [ ]:
from sklearn.metrics import mean_squared_error, matthews_corrcoef, confusion_matrix, f1_score
from scipy.stats import pearsonr


def predict(model, tokenizer, prefix="", final=False):
    model.eval()
    
    # Loop to handle MNLI double evaluation (matched, mis-matched)
    eval_output_dir = args['output_dir']

    results = {}
    EVAL_TASK = args['task_name']

    eval_dataset = load_and_cache_examples(EVAL_TASK, tokenizer)
    if not os.path.exists(eval_output_dir):
        os.makedirs(eval_output_dir)


    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=args['eval_batch_size'])

    # Eval!
    logger.info("***** Running evaluation {} *****".format(prefix))
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", args['eval_batch_size'])
    nb_eval_steps = 0
    binary_preds = None
    level_preds = None
    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        batch = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                      'token_type_ids': batch[2] if args['model_type'] in ['bert', 'xlnet'] else None,
                     }
            outputs = model(**inputs)
            binary_logits, level_logit = outputs[:3]
        nb_eval_steps += 1
        if binary_preds is None:
            binary_preds = binary_logits.detach().cpu().numpy()
            # level_preds = level_logit.detach().cpu().numpy()
        else:
            binary_preds = np.append(binary_preds, binary_logits.detach().cpu().numpy(), axis=0)
            # level_preds = np.append(level_preds, level_logit.detach().cpu().numpy(), axis=0)
    
    binary_preds = np.argmax(binary_preds, axis=1)
    # level_preds = np.squeeze(level_preds)
    
    # pdb.set_trace()
    
    return binary_preds, level_preds

In [ ]:
if args['do_predict']:
    print ('\n\n')
    global_step = 200
    print(f"loading model from {args['output_dir'] + '/checkpoint-{}/'.format(str(global_step)) + WEIGHTS_NAME}")
    checkpoint = args['output_dir'] + '/checkpoint-{}/'.format(str(global_step)) + WEIGHTS_NAME
    model = ClassificationModel(base_model=base_model)
    model.load_state_dict(torch.load(checkpoint))
    model.to(device)
    binary_preds, _ = predict(model, tokenizer, prefix=global_step)
    
    # pdb.set_trace()
    import pandas as pd
    import csv
    df = pd.read_csv(os.path.join(HOME, 'common_data/df_to_label3_labeled.csv'), index_col=0)
    label_name = 'Relations_binary'
    assert label_name.split("_")[0] == dimension.split("_")[0]
    df[label_name] = binary_preds
    print(df.columns)
    df.to_csv(os.path.join(HOME, 'common_data/df_to_label3_labeled.csv'), quoting=csv.QUOTE_NONNUMERIC)




loading model from ./drive/MyDrive/tvd5455/VvsNV/Relations/outputs_roberta//checkpoint-200/pytorch_model.bin


INFO:root:Creating features from dataset file at ./drive/MyDrive/tvd5455/VvsNV/common_data/
INFO:root:There are 891041 examples....
INFO:root:@USER cant wait til he runs for potus and theres 6000 families with a story to tell about a loved 1 murdered during covid . andy " the elderly slayer " cuomo who thinks hes king
INFO:root:Saving features into cached file ./drive/MyDrive/tvd5455/VvsNV/Relations/processed_data/cached_df_to_label3_roberta-large_128_both
INFO:root:***** Running evaluation 200 *****
INFO:root:  Num examples = 891041
INFO:root:  Batch size = 150
Evaluating: 100%|██████████| 5941/5941 [2:52:09<00:00,  1.74s/it]


Index(['TweetID', 'Tweet', 'ID', 'Text', 'Thoughts_binary',
       'Information_binary', 'Feelings_binary', 'Intimacy_binary',
       'Relations_binary'],
      dtype='object')


In [ ]:
import pandas as pd
dd = pd.read_csv(os.path.join(HOME, 'common_data/df_to_label1_labeled.csv'), index_col=0)
print(dd.columns)
print(dd.shape)

Index(['TweetID', 'Tweet', 'ID', 'Text', 'Thoughts_binary',
       'Information_binary', 'Feelings_binary', 'Intimacy_binary',
       'Relations_binary'],
      dtype='object')
(292412, 9)
